<a href="https://colab.research.google.com/github/riccardomarin/SpectralShapeAnalysis/blob/master/Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

--2020-07-26 14:50:29--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M  87.7MB/s    in 0.6s    

2020-07-26 14:50:30 (87.7 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.

In [11]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import meshplot
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans



In [19]:
v_src, f_src = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_043.off"))
plot(v_src, f_src)

L = igl.cotmatrix(v_src, f_src)
M = igl.massmatrix(v_src, f_src, igl.MASSMATRIX_TYPE_VORONOI)

evals_src, evecs_src = eigs(L,k=50,M=M,sigma=0)
evecs_src = evecs_src.astype(np.float32)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0282628…

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


# WKS

Wave Kernel Signature Descriptor

$K_E(x,x) = \sum\limits_{l=1}^{\infty}e^{- \frac{(log(E) - log(\lambda_l))^2}{2\sigma^2}} \phi_l(x)^2 $

In [16]:
def WKS(vertices, Lambda, Phi, Area, wks_size = 100):
    import numpy as np

    n_evecs = Phi.shape[1]

    D = np.matmul(Phi.T, Area.__matmul__(Phi**2))

    absoluteEigenvalues = np.abs(Lambda)
    emin = np.log(absoluteEigenvalues[1])
    emax = np.log(absoluteEigenvalues[len(absoluteEigenvalues) - 1])
    s = 7 * (emax - emin) / wks_size
    emin = emin + 2 * s
    emax = emax - 2 * s
    es = np.linspace(emin, emax, wks_size)

    T = np.exp(-np.square(
        np.tile(np.reshape(np.log(absoluteEigenvalues), (n_evecs, 1)), (1, wks_size)) - np.tile(es.T, (n_evecs, 1))) / (
                       2 * np.square(s)))
    WKS = np.matmul(D,T)
    WKS = np.dot(Phi, WKS)

    return np.asarray(WKS)

In [20]:
WKS(v_src, evals_src, evecs_src, M, wks_size = 100)

array([[ 0.51914734,  0.60065746,  0.7020671 , ..., 15.09456801,
        15.94336472, 16.70140092],
       [ 0.50662634,  0.58544659,  0.68366722, ..., 15.39509459,
        16.24171802, 16.99069736],
       [ 0.50306031,  0.58109541,  0.67838096, ..., 15.45456533,
        16.30471879, 17.0564127 ],
       ...,
       [ 0.45241686,  0.52049602,  0.60595213, ..., 15.9707666 ,
        16.83198927, 17.59020275],
       [ 0.45386049,  0.52223597,  0.60804422, ..., 15.96258041,
        16.82458395, 17.58384073],
       [ 0.45329893,  0.52155584,  0.60722378, ..., 15.96715284,
        16.82806494, 17.58604018]])